[![GitHub Repository](https://img.shields.io/badge/GitHub-Repository-181717?style=for-the-badge&logo=GitHub&link=https://github.com/Mearman/openalex-docs/tree/develop)](https://github.com/Mearman/openalex-docs/tree/develop)[![Open in GitHub](https://img.shields.io/badge/Open%20in-GitHub-181717?style=for-the-badge&logo=github&link=https://github.com/Mearman/openalex-docs/blob/develop/api-entities/authors/limitations.ipynb)](https://github.com/Mearman/openalex-docs/blob/develop/api-entities/authors/limitations.ipynb)[![Open in Colab](https://img.shields.io/badge/Open%20in-Colab-F9AB00?style=for-the-badge&logo=Google%20Colab&link=https://colab.research.google.com/github/Mearman/openalex-docs/blob/develop/api-entities/authors/limitations.ipynb)](https://colab.research.google.com/github/Mearman/openalex-docs/blob/develop/api-entities/authors/limitations.ipynb)

In [ ]:
%pip install --upgrade "git+https://github.com/Mearman/openalex-python-pydantic-v1.git"
%pip install pandasai

In [ ]:
import json
import pandas as pd
import numpy as np
from openalex_api import Configuration, ApiClient, AutocompleteApi, AuthorsApi, ConceptsApi, FundersApi, InstitutionsApi, PublishersApi, SourcesApi, WorksApi

configuration = Configuration(host="https://api.openalex.org")
autocomplete_api = AutocompleteApi(ApiClient(configuration))
authors_api = AuthorsApi(ApiClient(configuration))
concepts_api = ConceptsApi(ApiClient(configuration))
funders_api = FundersApi(ApiClient(configuration))
institutions_api = InstitutionsApi(ApiClient(configuration))
publishers_api = PublishersApi(ApiClient(configuration))
sources_api = SourcesApi(ApiClient(configuration))
works_api = WorksApi(ApiClient(configuration))

from pandasai import SmartDataframe
from pandasai.llm import OpenAI

In [ ]:
# @title  { run: "auto", display-mode: "form" }
openapi_token = "" # @param {type:"string"}

# Limitations

## Works with more than 100 authors are truncated

When retrieving a list of works in the API, the `authorships` list within each work will be cut off at 100 authorships objects in order to keep things running well. When this happens the boolean value `is_authors_truncated` will be available and set to `true`. This affects a small portion of OpenAlex, as there are around 35,000 works with more than 100 authors. This limitation does not apply to the [data snapshot](./../../download-all-data/openalex-snapshot.md).

* Example list of works with truncated authors
  [`https://api.openalex.org/works?filter=authors_count:>100`](https://api.openalex.org/works?filter=authors_count:%3E100)

In [ ]:
# @title { run: "auto", vertical-output: false }
# https://api.openalex.org/works?filter=authors_count:>100
filter="authors_count:>100" # @param {type: "string"}

response = works_api.get_works(
	filter=filter
)

df = pd.DataFrame(response.results)
display(df)

numeric_df = df[['id', 'display_name'] +
	[col for col in df.columns if df[col].dtype in ['int64', 'float64'] and col != 'relevance_score']]
display(numeric_df)

try:
	prompt = "Visualize this data" # @param {type:"string"}
	SmartDataframe(
		numeric_df,
		config={"llm": (OpenAI(api_token=openapi_token))},
	).chat(prompt)
except:
	if not openapi_token:
		print("Error: openapi_token not set")
	else:
		print("Error when creating SmartDataframe")

To see the full list of authors, go to the individual record for the work, which is never truncated.

* Work with all 249 authors available
  [`https://api.openalex.org/works/W2168909179`](https://api.openalex.org/works/W2168909179)

In [ ]:
# @title { run: "auto", vertical-output: false }
# https://api.openalex.org/works/W2168909179
id="W2168909179" # @param {type: "string"}

response = works_api.get_work(
	id=id
)

df = pd.DataFrame(response).T.rename(columns=lambda x: x[0]).drop(0).set_index('id')
display(df)

This affects filtering as well. So if you filter works using an author ID or ROR, you will not receive works where that author is listed further than 100 places down on the list of authors. We plan to change this in the future, so that filtering works as expected.